In [1]:
from operator import add

In [2]:
rdd = sc.textFile('citibike.csv')

In [3]:
record = rdd.map(lambda line: line.split(','))
totalCount = len(record.collect())
tripDurations = record.map(lambda x: (int(x[2]), 1)).reduceByKey(add)

529


In [4]:
medianCount = 0
for duration, count in sorted(tripDurations.collect()):
    medianCount += count
    if medianCount>=totalCount/2:
        print duration
        break

529
